In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, date_format, round, count, col, to_timestamp

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607282582075_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark=SparkSession.builder.appName('data_analytics').getOrCreate()
df=spark.read.csv('s3://covid19-bd-bucket/data/curated/Casos-COVID_19-Colombia/*.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: timestamp (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: boolean (nullable = true)
 |-- Fecha de inicio de síntomas: timestamp (nullable = true)
 |-- Fecha de muerte: timestamp (nullable = true)
 |-- Fecha de diagnóstico: timestamp (nullable = true)
 |-- Fecha de recuperación: timestamp (nullable = t

In [4]:
# shape
print((df.count(), len(df.columns)))
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1343322, 23)
+-------+------------------+----------------------------+-------------------+-------------------------+----------------+------------------+------------------------+-------+----------------+------------------+---------+-------------------+---------------+--------------------+------------------+-----------------------+
|summary|        ID de caso|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|              Edad|Unidad de medida de edad|   Sexo|Tipo de contagio|Ubicación del caso|   Estado|Código ISO del país|Nombre del país|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-------+------------------+----------------------------+-------------------+-------------------------+----------------+------------------+------------------------+-------+----------------+------------------+---------+-------------------+---------------+--------------------+------------------+-----------------------+
|  count|           1343322| 

In [5]:
# preparar sistema para eliminar datos para poder sobreescribir
URI = sc._gateway.jvm.java.net.URI 
Path = sc._gateway.jvm.org.apache.hadoop.fs.Path 
FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem 

fs = FileSystem.get(URI("s3n://covid19-bd-bucket"), sc._jsc.hadoopConfiguration())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# exploratory-data-analysis

In [6]:
# Aquí se va a listar la cantidad de contagios por mes
fecha_df = df.select('Fecha de diagnóstico')
contagios_por_mes = fecha_df.groupBy(date_format(df['Fecha de diagnóstico'], 'y-M').alias('Fecha de diagnóstico'))
contagios_por_mes = contagios_por_mes.count()
contagios_por_mes = contagios_por_mes.withColumn('Fecha de diagnóstico', to_timestamp(contagios_por_mes['Fecha de diagnóstico'], "y-M"))
contagios_por_mes = contagios_por_mes.orderBy('Fecha de diagnóstico')
contagios_por_mes.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/contagios_por_mes"))
contagios_por_mes.write.save("s3://covid19-bd-bucket/data/refined/contagios_por_mes", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|Fecha de diagnóstico| count|
+--------------------+------+
|                null|  4289|
| 2020-03-01 00:00:00|   908|
| 2020-04-01 00:00:00|  5592|
| 2020-05-01 00:00:00| 23014|
| 2020-06-01 00:00:00| 68705|
| 2020-07-01 00:00:00|216333|
| 2020-08-01 00:00:00|315543|
| 2020-09-01 00:00:00|220478|
| 2020-10-01 00:00:00|247286|
| 2020-11-01 00:00:00|235081|
| 2020-12-01 00:00:00|  6093|
+--------------------+------+

In [7]:
# Aquí se va a listar la cantidad de recuperaciones por mes
fecha_recuperacion_df = df.select('Fecha de recuperación')
recuperaciones_por_mes = fecha_recuperacion_df.groupBy(date_format(df['Fecha de recuperación'], 'y-M').alias('Fecha de recuperación'))
recuperaciones_por_mes = recuperaciones_por_mes.count()
recuperaciones_por_mes = recuperaciones_por_mes.withColumn('Fecha de recuperación', to_timestamp(recuperaciones_por_mes['Fecha de recuperación'], "y-M"))
recuperaciones_por_mes = recuperaciones_por_mes.orderBy('Fecha de recuperación')
recuperaciones_por_mes.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/recuperaciones_por_mes"))
recuperaciones_por_mes.write.save("s3://covid19-bd-bucket/data/refined/recuperaciones_por_mes", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+------+
|Fecha de recuperación| count|
+---------------------+------+
|                 null|110207|
|  2020-03-01 00:00:00|   201|
|  2020-04-01 00:00:00|  2046|
|  2020-05-01 00:00:00|  8459|
|  2020-06-01 00:00:00| 33648|
|  2020-07-01 00:00:00|109382|
|  2020-08-01 00:00:00|299994|
|  2020-09-01 00:00:00|282372|
|  2020-10-01 00:00:00|226452|
|  2020-11-01 00:00:00|247693|
|  2020-12-01 00:00:00| 22868|
+---------------------+------+

In [8]:
# Nivel de contagio por departamento
departamento_df = df.select('Nombre departamento')
contagios_por_departamento = departamento_df.groupBy(col('Nombre departamento').alias('Departamento'))
contagios_por_departamento = contagios_por_departamento.count()
contagios_por_departamento = contagios_por_departamento.orderBy(col('count').desc())
contagios_por_departamento.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/contagios_por_departamento"))
contagios_por_departamento.write.save("s3://covid19-bd-bucket/data/refined/contagios_por_departamento", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------+
|   Departamento| count|
+---------------+------+
|         BOGOTA|382704|
|      ANTIOQUIA|218131|
|          VALLE|111017|
|      SANTANDER| 54535|
|   CUNDINAMARCA| 53678|
|   BARRANQUILLA| 45922|
|      ATLANTICO| 33151|
|      CARTAGENA| 32299|
|          HUILA| 30563|
|         TOLIMA| 30528|
|          CESAR| 30177|
|NORTE SANTANDER| 29911|
|           META| 28764|
|        CORDOBA| 28438|
|         CALDAS| 25921|
|         NARIÑO| 25882|
|      RISARALDA| 24034|
|         BOYACA| 22411|
|        QUINDIO| 17724|
|          SUCRE| 16653|
+---------------+------+
only showing top 20 rows

In [9]:
# Nivel de contagio por edad
edad_df = df.select('Edad')
contagio_por_edad = edad_df.groupBy(col('Edad'))
contagio_por_edad = contagio_por_edad.count()
contagio_por_edad = contagio_por_edad.orderBy(col('count').desc())
contagio_por_edad.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/contagio_por_edad"))
contagio_por_edad.write.save("s3://covid19-bd-bucket/data/refined/contagio_por_edad", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Edad|count|
+----+-----+
|  30|35470|
|  29|35069|
|  28|35031|
|  27|34207|
|  26|33989|
|  31|33944|
|  25|32545|
|  32|32527|
|  33|31822|
|  35|31640|
|  34|31599|
|  24|30696|
|  36|30014|
|  38|29540|
|  37|29230|
|  39|28358|
|  40|28041|
|  23|27849|
|  41|25728|
|  22|24890|
+----+-----+
only showing top 20 rows

In [10]:
# Tipo de contagio
tipo_contagio_df = df.select('Tipo de contagio')
tipo_de_contagio = tipo_contagio_df.groupBy(col('Tipo de contagio'))
tipo_de_contagio = tipo_de_contagio.count()
tipo_de_contagio.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/tipo_de_contagio"))
tipo_de_contagio.write.save("s3://covid19-bd-bucket/data/refined/tipo_de_contagio", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------+
|Tipo de contagio|  count|
+----------------+-------+
|     RELACIONADO| 121921|
|       IMPORTADO|    918|
|      EN ESTUDIO|1220483|
+----------------+-------+

In [12]:
# Contagio segun sexo
sexo_df = df.select('Sexo')
contagio_segun_sexo = sexo_df.groupBy(col('Sexo'))
contagio_segun_sexo = contagio_segun_sexo.count()
contagio_segun_sexo.show()

# guardado de datos nuevos
fs.delete(Path("s3n://covid19-bd-bucket/data/refined/contagio_segun_sexo"))
contagio_segun_sexo.coalesce(1).write.save("s3://covid19-bd-bucket/data/refined/contagio_segun_sexo", format='csv', header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|Sexo| count|
+----+------+
|   M|668803|
|   F|674519|
+----+------+